In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ridge_regression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression,BayesianRidge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data=pd.read_csv("../input/train.csv")
test_data=pd.read_csv("../input/test.csv")
submission=pd.read_csv("../input/sample_submission.csv")

train_data=train_data[train_data["total_pax"]<15]

target=train_data.amount_spent_per_room_night_scaled
train_data.drop(columns=['amount_spent_per_room_night_scaled'],inplace=True)
test_train_data=pd.concat([train_data,test_data],axis=0)

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data["state_code_residence"].value_counts()

In [ ]:
catboost=CatBoostRegressor(task_type = "GPU",eval_metric='RMSE',random_state=42,loss_function='RMSE',iterations=2000,depth=10)
categorical_features_indices=[0,1,2,3,4,7,8,9,10,12,13,14,16,17,18,19,20,27,28,29,30,31]

test_train_data=pd.concat([train_data,test_data],axis=0)
test_train_data.drop(columns=["memberid","reservation_id","reservationstatusid_code"],inplace=True)
test_train_data["season_holidayed_code"]=test_train_data["season_holidayed_code"].fillna(5)
test_train_data["state_code_residence"]=test_train_data["state_code_residence"].fillna(39)

test_train_data["checkin_date_weekday"]=test_train_data["checkout_date"].apply(lambda x:int(datetime.strptime(x, '%d/%m/%y').strftime("%w")))
test_train_data["total_people*night"]=(test_train_data["total_pax"])*test_train_data["roomnights"]
test_train_data["difference_booking_checkin"]=(test_train_data["checkin_date"].apply(lambda x:datetime.strptime(x, '%d/%m/%y'))-
                                       test_train_data["booking_date"].apply(lambda x:datetime.strptime(x,'%d/%m/%y'))).dt.days
test_train_data["difference_checkout_checkin"]=(test_train_data["checkout_date"].apply(lambda x:datetime.strptime(x, '%d/%m/%y'))-
                                       test_train_data["checkin_date"].apply(lambda x:datetime.strptime(x,'%d/%m/%y'))).dt.days
test_train_data["difference_checkout_booking"]=(test_train_data["checkout_date"].apply(lambda x:datetime.strptime(x, '%d/%m/%y'))-
                                       test_train_data["booking_date"].apply(lambda x:datetime.strptime(x,'%d/%m/%y'))).dt.days


test_train_data["total_people"]=test_train_data["numberofadults"]+test_train_data["numberofchildren"]
test_train_data["total_people*diff"]=(test_train_data["total_people"])*test_train_data["difference_checkout_checkin"]
test_train_data["year"]=test_train_data["checkin_date"].apply(lambda x:int(x.split("/")[2]))
test_train_data["month"]=test_train_data["checkin_date"].apply(lambda x:int(x.split("/")[1]))
test_train_data["checkout_yearday"]=test_train_data["checkout_date"].apply(lambda x:int(datetime.strptime(x, '%d/%m/%y').strftime("%j")))
test_train_data["checkin_yearday"]=test_train_data["checkin_date"].apply(lambda x:int(datetime.strptime(x, '%d/%m/%y').strftime("%j")))                                                                        
# test_train_data["checkout_yearweek"]=test_train_data["checkout_date"].apply(lambda x:int(datetime.strptime(x, '%d/%m/%y').strftime("%W")))
# test_train_data["checkin_yearweek"]=test_train_data["checkin_date"].apply(lambda x:int(datetime.strptime(x, '%d/%m/%y').strftime("%W"))) 
test_train_data["temp2"]=LabelEncoder().fit_transform(test_train_data["cluster_code"])
test_train_data["resort_main_product"]=test_train_data["main_product_code"]*(1+test_train_data["temp2"])
test_train_data["mix45"]=test_train_data["main_product_code"]*test_train_data["difference_checkout_checkin"]
test_train_data=pd.concat([test_train_data,pd.get_dummies(test_train_data["mix45"])],axis=1)

test_train_data.drop(columns=["temp2"],inplace=True)

X_train=test_train_data[0:341418]
y=target
X_test=test_train_data[341418:]
# X_train=X_temp[X_temp["difference_booking_checkin"]>=0]
# y=y[X_temp["difference_booking_checkin"]>=0]


In [ ]:
catboost.fit(X_train,y,cat_features=categorical_features_indices)
pred=catboost.predict(X_test)
submission["amount_spent_per_room_night_scaled"]=pred
submission.to_csv("cb22.csv",index=False)

In [ ]:
# from IPython.display import HTML
# def create_download_link(title = "Download CSV file", filename = "data.csv"):  
#     html = '<a href={filename}>{title}</a>'
#     html = html.format(title=title,filename=filename)
#     return HTML(html)

# # create a link to download the dataframe which was saved with .to_csv method
# create_download_link(filename='cb2.csv')